<a href="https://colab.research.google.com/github/Alfonso-Jesus-Garcia-Moya/Tecnicas_de_inteligencia_artificial/blob/main/SEMANA%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 16. Reposición

## Particle Swarm Optimization - KNN

### Práctica guiada

Problema de maximización : encontrar el valor más alto para incrementar (métrica de evaluación, probabilidad, ganancia, beneficio, energía producida, estabilidad)

Problema de minimización : encontrar el valor más bajo para reducir (error, coste, tiempo, pérdida, disntancia, energía consumida)

In [ ]:
#importanto librerías
import random
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [ ]:
#cargando datos
df = load_breast_cancer()

In [ ]:
#dividiendo en x/y
X = df.data
y = df.target

In [ ]:
#dividiendo datos en entrenamiento y prubea
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#número de columnas, para que el hiperparámetro no sea mayor que el número total de las columnas
n_features = X.shape[1]   #necesario para max_features

Inercia: parte del movimiento que permite a la partícula seguir viajando en la misma dirección en la que se movía (impulso)

Peso cognitivo: tendencia de la partícula a regresar a su propia mejor exploración personal (pbest) (memoria individual)

Peso social: tendencia a moverse hacie el mejor lugar encontrado por el grupo completo (gbest) (aprendizaje colectivo)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
import numpy as np
import random

def evaluar_knn(n_neighbors_real):
    #n_neighbors_real: valor continuo que convertiremos a entero [1, 50]

    #redondeando y asegurando rangos válidos para n_neighbors
    n_neighbors = int(round(n_neighbors_real))
    n_neighbors = max(1, min(50, n_neighbors))   #ejemplo: de 1 a 50

    #creando modelo K-NN con hiperparámetros sugeridos por la partícula
    clf = KNeighborsClassifier(
        n_neighbors=n_neighbors
    )

    #entrenando K-NN
    clf.fit(X_train, y_train)

    #predicciones
    y_pred = clf.predict(X_test)

    #métrica de evaluación
    f1 = f1_score(y_test, y_pred, average="macro")
    return f1


#configuración PSO para K-NN
num_particles = 10
num_iter = 30

W = 0.6
C1 = 1.2
C2 = 1.2

#rangos para los parámetros a optimizar
n_neighbors_min, n_neighbors_max = 1, 50   #pueden ajustar este rango

particles = []

#población inicial
for _ in range(num_particles):
    #posición inicial aleatoria dentro del rango permitido
    x1 = random.uniform(n_neighbors_min, n_neighbors_max)   #n_neighbors

    #velocidades iniciales
    v1 = random.uniform(-2, 2)

    #fitness inicial, evaluando K-NN con los hiperparámetros
    val = evaluar_knn(x1)

    particles.append({
        "pos": np.array([x1], dtype=float),
        "vel": np.array([v1], dtype=float),
        "best_pos": np.array([x1], dtype=float),
        "best_val": val
    })

#mejor global inicial
global_best = max(particles, key=lambda p: p["best_val"])
global_best_pos = global_best["best_pos"].copy()
global_best_val = global_best["best_val"]

#PSO
for it in range(num_iter):
    for p in particles:
        r1 = random.random()
        r2 = random.random()

        #actualizando velocidad
        p["vel"] = (
            W * p["vel"]
            + C1 * r1 * (p["best_pos"] - p["pos"])
            + C2 * r2 * (global_best_pos - p["pos"])
        )

        #actualizando posición
        p["pos"] = p["pos"] + p["vel"]

        #limitando la posición dentro del rango
        p["pos"][0] = np.clip(p["pos"][0], n_neighbors_min, n_neighbors_max)

        #evaluando nuevo desempeño
        val = evaluar_knn(p["pos"][0])

        #actualizando mejor personal
        if val > p["best_val"]:
            p["best_val"] = val
            p["best_pos"] = p["pos"].copy()

        #actualizando mejor global
        if val > global_best_val:
            global_best_val = val
            global_best_pos = p["pos"].copy()

    #decodificando parámetros "reales" para imprimir
    best_n_neighbors = int(round(global_best_pos[0]))
    best_n_neighbors = max(1, min(50, best_n_neighbors))


    print(
        f"Iter {it:2d} | Mejor F1 global = {global_best_val:.4f} "
        f"| Parámetros = n_neighbors={best_n_neighbors}'"
    )


Iter  0 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  1 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  2 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  3 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  4 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  5 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  6 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  7 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  8 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter  9 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 10 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 11 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 12 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 13 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 14 | Mejor F1 global = 0.9776 | Parámetros = n_neighbors=11'
Iter 15 | 

In [ ]:
print("Mejores hiperparámetros encontrados por PSO:")
print("n_neighbors        =", best_n_neighbors)
print("F1 final   =", global_best_val)

Mejores hiperparámetros encontrados por PSO:
n_neighbors        = 11
F1 final   = 0.9776014201430585


## Código implementado

In [ ]:
#importando librerías
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score

In [ ]:
#cargando el conjunto de datos
df = load_breast_cancer()

In [ ]:
#obteniendo información del conjunto
df_features = pd.DataFrame(df.data, columns = df.feature_names)
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [ ]:
df_features

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [ ]:
#obteniendo primeros registros
df_features.head()
#df_features.tail()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
#obteniendo información
df_features.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [ ]:
#obteniendo variable predictora
df.target_names

array(['malignant', 'benign'], dtype='<U9')

In [ ]:
#obteniendo registros por etiqueta (maligno[0], benigno[1])
df_target = pd.DataFrame(df.target, columns=['target'])
df_target['target'].value_counts()

,count
target,
1,357
0,212


In [ ]:
#dividiendo en x/y
X, y = df.data, df.target

In [ ]:
#dividiendo los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#creando clasificador
clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=11)

In [ ]:
#evaluando con accuracy
accuracy = clf.score(X_test, y_test)

y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred)

print("Exactitud:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)

Exactitud: 0.9790209790209791
Precision: 0.9794549266247379
F1-score: 0.9832402234636871


In [ ]:
#evaluando con accuracy (primera ejecución)
accuracy = clf.score(X_test, y_test)

y_pred = clf.predict(X_test)

precision = precision_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred)

print("Exactitud:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)

Exactitud: 0.9790209790209791
Precision: 0.9794549266247379
F1-score: 0.9832402234636871
